In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\animal classifier\\Animal-classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\animal classifier\\Animal-classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:

    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        #prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "catsndogs")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None
        self.training_set = None
        self.test_set = None    
 
    def train_valid_generator(self):

        train_datagen_kwargs = dict(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
        )

        training_set_kwargs = dict(
                            
                                target_size = (64, 64),
                                batch_size=self.config.params_batch_size,
                                class_mode = 'binary'
        )
        test_datagen_kwargs = dict(rescale = 1./255)

        test_set_kwargs = dict(
                            target_size = (64, 64),
                            batch_size=self.config.params_batch_size,
                            class_mode = 'binary')


        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **test_datagen_kwargs
        )

        self.test_set = test_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **test_set_kwargs
        )
        


        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                **train_datagen_kwargs
            )
        else:
            train_datagen = test_datagen

        self.training_set = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **training_set_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

  

    def train(self):
        self.model = tf.keras.models.Sequential()
        self.model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=self.config.params_image_size))
        self.model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
        self.model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
        self.model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        self.model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        
        self.model.fit(x=self.training_set, validation_data=self.test_set, epochs=self.config.params_epochs)

        self.save_model(
                    path=self.config.trained_model_path,
                    model=self.model
                    )

    



In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.train_valid_generator()
    training.train()

    

except Exception as e:
    raise e

[2023-07-08 14:21:22,804: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-08 14:21:22,838: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-08 14:21:22,846: INFO: common: created directory at: artifacts]
[2023-07-08 14:21:22,852: INFO: common: created directory at: artifacts\training]


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'artifacts\\data_ingestion\\catsndogs'